# Tutorial of Ising model & QUBO model

## 初期設定
### ライブラリのインポート

In [1]:
import numpy as np
import plotly.express as px
np.set_printoptions(edgeitems=10)

### 各種項目の設定
- スピン数 : `num_spin`
- 相互作用係数/外部磁場の最小値 : `coef_min`
- 相互作用係数/外部磁場の最大値 : `coef_max`

を設定する．  
実際に動かして確認する際には，下記セルの値を変更して動かしてみてください．

In [2]:
num_spin = 5
coef_min = 0
coef_max = 9

## イジングモデル
イジングモデルのエネルギー関数 $\mathcal{H}$ は，次の式(1)で表される．
$$
\begin{align}
    s_i &=
        \begin{cases}
            1 \\
            -1 \\
        \end{cases} \\
\mathcal{H} &= \sum_{(i,j) \in E} J_{i,j} s_i s_j + \sum_{i \in V} h_i s_i \tag{1}
\end{align}
$$

数式で見ても分かりづらいので，実際に中身を見ていこう．  
まず，スピン $\boldsymbol{s}$ の状態をランダムに設定する．

In [3]:
s = np.random.randint(0, 2, size=num_spin)
s = np.where(s == 0, -1, s)
print(f"s = {s}")

s = [ 1 -1  1 -1  1]


次に，スピン $\boldsymbol{s}$ におけるイジングモデル $J_{\mathrm{ising}}$ をランダムに作成する．

In [4]:
J_ising = np.zeros((num_spin, num_spin), dtype=np.int16)
for i in range(num_spin):
    J_ising[i, i:] = np.random.randint(coef_min, coef_max+1, size=num_spin-i)
print(f"J_ising = \n{J_ising}")

J_ising = 
[[5 3 9 3 5]
 [0 8 4 3 9]
 [0 0 8 0 7]
 [0 0 0 4 4]
 [0 0 0 0 5]]


これで準備は終わり．  
ハミルトニアン $\mathcal{H}\mathrm{ising}$ の値を計算してみよう．  
計算方法は以下の2通り．
1. 式(1)を展開して，1個ずつ丁寧に足していく
2. 行列計算を用いる

まずは，  
`1. 式(1)を展開して，1個ずつ丁寧に足していく`  
方法でやってみる．

In [5]:
# 式(1)第1項：相互作用係数の計算
H1_ising = 0
for i in range(num_spin):
    for j in range(i+1, num_spin):
        H1_ising += J_ising[i,j] * s[i] * s[j]
        print(f"J_{i}{j} * s_{i} * s_{j} = {J_ising[i,j]} * {s[i]} * {s[j]} = {J_ising[i,j] * s[i] * s[j]} -> Total: {H1_ising}")
print(f"H = {H1_ising}, \n")

# 式(1)第2項：外部磁場の計算
for i in range(num_spin):
    H1_ising += J_ising[i,i] * s[i]
    print(f"h_{i} * s_{i} = {J_ising[i,i]} * {s[i]} = {J_ising[i,i] * s[i]} -> Total: {H1_ising}")
print(f"H = {H1_ising}\n")

print(f"-> The Hamiltonian is {H1_ising}.")

J_01 * s_0 * s_1 = 3 * 1 * -1 = -3 -> Total: -3
J_02 * s_0 * s_2 = 9 * 1 * 1 = 9 -> Total: 6
J_03 * s_0 * s_3 = 3 * 1 * -1 = -3 -> Total: 3
J_04 * s_0 * s_4 = 5 * 1 * 1 = 5 -> Total: 8
J_12 * s_1 * s_2 = 4 * -1 * 1 = -4 -> Total: 4
J_13 * s_1 * s_3 = 3 * -1 * -1 = 3 -> Total: 7
J_14 * s_1 * s_4 = 9 * -1 * 1 = -9 -> Total: -2
J_23 * s_2 * s_3 = 0 * 1 * -1 = 0 -> Total: -2
J_24 * s_2 * s_4 = 7 * 1 * 1 = 7 -> Total: 5
J_34 * s_3 * s_4 = 4 * -1 * 1 = -4 -> Total: 1
H = 1, 

h_0 * s_0 = 5 * 1 = 5 -> Total: 6
h_1 * s_1 = 8 * -1 = -8 -> Total: -2
h_2 * s_2 = 8 * 1 = 8 -> Total: 6
h_3 * s_3 = 4 * -1 = -4 -> Total: 2
h_4 * s_4 = 5 * 1 = 5 -> Total: 7
H = 7

-> The Hamiltonian is 7.


次に，  
`2. 行列計算を用いる`

方法でやってみる．  
式(1)は，次の式(2)のように書き換えることができる．
$$
\begin{align}
    \mathcal{H}_{\mathrm{qubo}} &= \sum_{(i,j) \in E} J_{i,j} s_i s_j + \sum_{i \in V} h_i s_i \\
    &= \boldsymbol{s}^T \left( J - diag(J) \right) \boldsymbol{s} + <diag(J), \boldsymbol{s}> \tag{3}
\end{align}
$$

ただし，
$$
J =
\begin{cases}
    J_{i,j} \quad \mathrm{if} ~ i \ne j \\
    h_i  \quad \mathrm{if} ~ i = j \\
\end{cases}
$$

したがって，式(3)に基づいて，エネルギー関数 $\mathcal{H}_{\mathrm{ising}}$ の値を計算する．

In [6]:
H2_ising = 0

H2_ising = int(np.dot(np.dot(s, np.triu(J_ising, k=1)), s)) + np.dot(s, np.diag(J_ising))
print(f"H_ising = {H2_ising}")

H_ising = 7


以上より，QUBOモデルにおいては，
1. 式(2)を展開して，1個ずつ丁寧に足していく
2. 式(2)を1つにまとめて，行列を用いて計算する

のどちらで計算してもハミルトニアンの値は変わらない．  


## QUBOモデル

QUBOモデルのエネルギー関数 $\mathcal{H}_{\mathrm{qubo}}$ は，次の式(1)で表される．
$$
\begin{align}
    x_i &=
        \begin{cases}
            1 \quad\\
            0 \quad\\
        \end{cases} \\
\mathcal{H}_{\mathrm{qubo}} &= \sum_{(i,j) \in E} J'_{i,j} x_i x_j + \sum_{i \in V} h'_i x_i + \mathrm{const} \tag{2}
\end{align}
$$

ただし，$\mathrm{const}$ は定数．  
数式で見ても分かりづらいので，実際に中身を見ていこう．  
まず，スピン $\boldsymbol{x}$ の状態をランダムに設定する．

In [7]:
x = np.random.randint(0, 2, size=num_spin)
print(f"x = {x}\n")

x = [1 1 1 0 0]



次に，スピン $\boldsymbol{x}$ におけるイジングモデル $J_{\mathrm{qubo}}$ をランダムに作成する．

In [8]:
J_qubo = np.zeros((num_spin, num_spin), dtype=np.int16)
for i in range(num_spin):
    J_qubo[i, i:] = np.random.randint(coef_min, coef_max+1, size=num_spin-i)
print(f"J_qubo = \n{J_qubo}")

J_qubo = 
[[7 8 3 3 5]
 [0 3 8 6 8]
 [0 0 0 3 3]
 [0 0 0 8 8]
 [0 0 0 0 4]]


これで準備は終わり．  
ハミルトニアン $\mathcal{H}_{\mathrm{qubo}}$ の値を計算してみよう．  
計算方法は以下の2通り．  
1. 式(2)を展開して，1個ずつ丁寧に足していく
2. 式(2)を1つにまとめて，行列を用いて計算する

まずは，  
`1. 式(2)を展開して，1個ずつ丁寧に足していく`  
方法でやってみる

In [9]:
# 式(2)-第1項：相互作用係数の計算
H1_qubo = 0
for i in range(num_spin):
    for j in range(i+1, num_spin):
        H1_qubo += J_qubo[i,j] * x[i] * x[j]
        print(f"J_{i}{j} * x_{i} * x_{j} = {J_qubo[i,j]} * {x[i]} * {x[j]} = {J_qubo[i,j] * x[i] * x[j]} -> Total: {H1_qubo}")
print(f"H = {H1_qubo}, \n")

# 式(1)-第2項：外部磁場の計算
for i in range(num_spin):
    H1_qubo += J_qubo[i,i] * x[i]
    print(f"h_{i} * x_{i} = {J_qubo[i,i]} * {x[i]} = {J_qubo[i,i] * x[i]} -> Total: {H1_qubo}")
print(f"H = {H1_qubo}\n")

print(f"-> H_qubo = {H1_qubo}.")

J_01 * x_0 * x_1 = 8 * 1 * 1 = 8 -> Total: 8
J_02 * x_0 * x_2 = 3 * 1 * 1 = 3 -> Total: 11
J_03 * x_0 * x_3 = 3 * 1 * 0 = 0 -> Total: 11
J_04 * x_0 * x_4 = 5 * 1 * 0 = 0 -> Total: 11
J_12 * x_1 * x_2 = 8 * 1 * 1 = 8 -> Total: 19
J_13 * x_1 * x_3 = 6 * 1 * 0 = 0 -> Total: 19
J_14 * x_1 * x_4 = 8 * 1 * 0 = 0 -> Total: 19
J_23 * x_2 * x_3 = 3 * 1 * 0 = 0 -> Total: 19
J_24 * x_2 * x_4 = 3 * 1 * 0 = 0 -> Total: 19
J_34 * x_3 * x_4 = 8 * 0 * 0 = 0 -> Total: 19
H = 19, 

h_0 * x_0 = 7 * 1 = 7 -> Total: 26
h_1 * x_1 = 3 * 1 = 3 -> Total: 29
h_2 * x_2 = 0 * 1 = 0 -> Total: 29
h_3 * x_3 = 8 * 0 = 0 -> Total: 29
h_4 * x_4 = 4 * 0 = 0 -> Total: 29
H = 29

-> H_qubo = 29.


次に，  
`2. 式(2)を1つにまとめて，行列を用いて計算する`  
方法でやってみる．  
式(2)は，次の式(3)のように書き換えることができる．
$$
\begin{align}
    \mathcal{H}_{\mathrm{qubo}} &= \sum_{(i,j) \in E} J'_{i,j} x_i x_j + \sum_{i \in V} h'_i x_i + \mathrm{const} \\
    &= -\boldsymbol{x}^T Q \boldsymbol{x} + \mathrm{const} \tag{3}
\end{align}
$$

式(3)に基づいて，エネルギー関数 $\mathcal{H}_{\mathrm{qubo}}$ の値を計算する．

In [10]:
H2_qubo = int(np.dot(np.dot(x, J_qubo), x))
print(f"H_qubo = {H2_qubo}")

H_qubo = 29


以上より，QUBOモデルにおいては，
1. 式(2)を展開して，1個ずつ丁寧に足していく
2. 式(2)を1つにまとめて，行列を用いて計算する

のどちらで計算してもハミルトニアンの値は変わらない．  

なぜ変わらないのかを，スピン数が4の場合のQUBOモデルで考えてみよう．

$$
\begin{align}
    \mathcal{H}_{\mathrm{qubo}} &=  \boldsymbol{x}^T Q \boldsymbol{x} \\
    &= \left[ x_1 ~ x_2 ~ x_3 ~ x_4 \right]
        \left[\begin{array}{cccc}
        q_{1,1} & q_{1,2} & q_{1,3} & q_{1,4} \\
        0       & q_{2,2} & q_{2,3} & q_{2,4} \\
        0       & 0       & q_{3,3} & q_{3,4} \\
        0       & 0       & 0       & q_{4,4} \\
        \end{array}\right]
        \left[\begin{array}{c}
            x_1 \\  x_2 \\ x_3 \\ x_4
        \end{array}\right] \\
    &= \left[ q_{1,1} x_1 ~~ q_{1,2} x_1 + q_{2,2} x_2 ~~ q_{1,3} x_1 + q_{2,3} x_2 + q_{3,3} x_3 ~~ q_{1,4} x_1 + q_{2,4} x_2 + q_{3,4} x_3 + q_{4,4} x_4 \right]
        \left[\begin{array}{c}
            x_1 \\  x_2 \\ x_3 \\ x_4
        \end{array}\right] \\
    &= q_{1,1} x_1^2 + q_{1,2} x_1 x_2 + q_{2,2} x_2^2 + q_{1,3} x_1 x_3 + q_{2,3} x_2 x_3 + q_{3,3} x_3^2 + q_{1,4} x_1 x_4 + q_{2,4} x_2 x_4 + q_{3,4} x_3 x_4 + q_{4,4} x_4^2 \\
    & (x_i = [0,1]よりx_i^2 = x_iであるから) \\
    &= q_{1,2} x_1 x_2 + q_{1,3} x_1 x_3 + q_{1,4} x_1 x_4 + q_{2,3} x_2 x_3 + q_{2,4} x_2 x_4 + q_{3,4} x_3 x_4 + q_{1,1} x_1 + q_{2,2} x_2 + q_{3,3} x_3 + q_{4,4} x_4 \\
    &= \sum_{i=1}^3 \sum_{j=i+1}^4 q_{i,j} x_i x_j + \sum_{i=1}^4 q_{i,i} x_i  \tag{4}
\end{align}
$$

したがって，

$$
\begin{align}
    \mathcal{H}_{\mathrm{qubo}} &=  \boldsymbol{x}^T Q \boldsymbol{x} \\
    &= \sum_{i>j} q_{i,j} x_i x_j + \sum_{i} q_{i,i} x_i \\
    & (q_{i,j}=J'_{i,j}, ~q_{i,i}=h'_iとすれば(：単に文字が違うだけで中身は同じ)) \\
    &= \sum_{(i,j) \in E} J'_{i,j} x_i x_j + \sum_{i \in V} h'_i x_i + \mathrm{const} \tag{5}
\end{align}
$$
ただし，$\mathrm{const}=0$である．  
したがって，式(2)と式(3)は等価なので，
1. 式(2)を展開して，1個ずつ丁寧に足していく
2. 式(2)を1つにまとめて，行列を用いて計算する

のどちらで計算しても良いことになる．  

ここで1つ，疑問に出てくるモノがある．  
**Q. 行列を用いる計算は，イジングモデルには適用できないのか？**  
**A. できない**  
なぜかというと，QUBOモデルでは$x_i=[0,1]$より，$x_i^2=x_i$と変換することができたが，イジングモデルでは$s_i=[-1,1]$であることから，$s_i^2 \ne s_i$だからである．  
つまり，イジングモデルにおけるハミルトニアンの計算では  
1. 式(2)を展開して，1個ずつ丁寧に足していく

という手法を用いて，地道に計算していくしかない．


## イジングモデルとQUBOモデルの変換

**イジングモデル -> QUBOモデルを考える**

$$
\begin{align}
    \mathcal{H}_{\mathrm{ising}} &= \sum_{(i,j) \in E} J_{i,j} s_i s_j + \sum_{i \in V} h_i s_i \\
    &= \sum_{(i,j) \in E} J_{i,j} (2 x_i - 1) (2 x_j - 1) + \sum_{i \in V} h_i (2 x_i - 1) \\
    &= \sum_{(i,j) \in E} J_{i,j} (4 x_i x_j - 2 x_i - 2 x_j + 1) + \sum_{i \in V} h_i (2 x_i - 1) \\
    &= \left( \sum_{(i,j) \in E} 4 J_{i,j} x_i x_j - \sum_{(i,j) \in E} 2 J_{i,j} x_i - \sum_{(i,j) \in E} 2 J_{i,j} x_j + \sum_{(i,j) \in E} J_{i,j} \right) + \left( \sum_{i \in V} 2 h_i x_i - \sum_{i \in V} h_i \right) \\
    &= \left( \sum_{(i,j) \in E} 4 J_{i,j} x_i x_j \right) + \left( - \sum_{(i,j) \in E} 2 J_{i,j} x_i - \sum_{(i,j) \in E} 2 J_{i,j} x_j + \sum_{i \in V} 2 h_i x_i \right) + \left(\sum_{(i,j) \in E} J_{i,j} - \sum_{i \in V} h_i \right) \\
    &= \sum_{(i,j) \in E} 4 J_{i,j} x_i x_j + \left( \sum_{i \in V} 2 h_i x_i - \sum_{(i,j) \in E} 2 J_{i,j} x_i - \sum_{(i,j) \in E} 2 J_{i,j} x_j \right) + \mathrm{const} \\
    &= \sum_{(i,j) \in E} 4 J_{i,j} x_i x_j + \left( \sum_{i \in V} 2 h_i x_i - \sum_{(i,j) \in E} 4 J_{i,j} x_i \right) + \mathrm{const} \\
    &= \sum_{(i,j) \in E} 4 J_{i,j} x_i x_j + \left\{ \sum_{i \in V} \left( 2 h_i - \sum_{j \in \delta_i} 4 J_{i,j} \right) x_i \right\} + \mathrm{const} \\
    &= \sum_{(i,j) \in E} J'_{i,j} x_i x_j - \sum_{i \in V} h'_i x_i + \mathrm{const} \tag{6}
\end{align}
$$

したがって，

$$
\begin{align}
    J'_{i,j} &= 4 J_{i,j} \\
    h'_i &= 2 h_i - \sum_{j \in \delta_i} 4 J_{i,j} \tag{7}
\end{align}
$$

**QUBOモデル -> イジングモデルを考える**
$$
\begin{align}
    \mathcal{H}_{\mathrm{qubo}} &= \sum_{(i,j) \in E} J'_{i,j} x_i x_j + \sum_{i \in V} h'_i x_i \\
    &= \sum_{(i,j) \in E} J'_{i,j} \frac{\left( s_i+1 \right)}{2} \frac{\left( s_j+1 \right)}{2} + \sum_{i \in V} h'_i \frac{\left( s_i+1 \right)}{2} \\
    &= \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} \left( s_i s_j + s_i + s_j + 1 \right) + \sum_{i \in V} \frac{h'_i}{2} \left( s_i + 1 \right)\\
    &= \left( \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i s_j + \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i + \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_j + \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} \right) + \left( \sum_{i \in V} \frac{h'_i}{2} s_i + \sum_{i \in V} \frac{h'_i}{2} \right) \\
    &= \left( \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i s_j \right) + \left( \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i + \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_j + \sum_{i \in V} \frac{h'_i}{2} s_i \right) + \left( \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} + \sum_{i \in V} \frac{h'_i}{2} \right) \\
    &= \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i s_j + \left( \sum_{i \in V} \frac{h'_i}{2} s_i + \sum_{(i,j) \in E} \frac{J'_{i,j}}{2} s_i \right) + \mathrm{const} \\
    &= \sum_{(i,j) \in E} \frac{J'_{i,j}}{4} s_i s_j + \sum_{i \in V} \left( \frac{h'_i}{2} + \sum_{j \in \delta_i} \frac{J'_{i,j}}{2} \right) s_i + \mathrm{const} \\
    &= \sum_{(i,j) \in E} J_{i,j} s_i s_j + \sum_{i \in V} h_i s_i \tag{8}
\end{align}
$$

したがって，

$$
\begin{align}
    J_{i,j} &= \frac{J'_{i,j}}{4} \\
    h_i &= \frac{h'_i}{2} + \sum_{j \in \delta_i} \frac{J'_{i,j}}{2} \tag{9}
\end{align}
$$

## QUBOモデルの形式による違い
QUBOモデルが
1. 上三角行列
2. 下三角行列
3. 対称行列

の場合で，ハミルトニアンを比較してみる．  
まずは，スピン $\boldsymbol{x}$ の状態をランダムに設定する．

In [11]:
x = np.random.randint(0, 2, size=num_spin)
print(f"x = {x}\n")

x = [1 0 0 1 0]



In [12]:
# ========== 上三角行列の場合 ==========
Q_upper = np.zeros((num_spin, num_spin), dtype=np.int16)
for i in range(num_spin):
    Q_upper[i, i:] = np.random.randint(coef_min, coef_max+1, size=num_spin-i)
print(f"Q_upper = \n{Q_upper}\n")

H_upper = int(np.dot(np.dot(x, Q_upper), x))
print(f"H = {H_upper}")

Q_upper = 
[[6 3 3 9 6]
 [0 6 7 6 4]
 [0 0 8 1 4]
 [0 0 0 0 2]
 [0 0 0 0 8]]

H = 15


In [13]:
# 2. 下三角行列の場合
Q_lower = Q_upper.T
print(f"Q_lower = \n{Q_lower}\n")

H_upper = int(np.dot(np.dot(x, Q_upper), x))
print(f"H = {H_upper}")

Q_lower = 
[[6 0 0 0 0]
 [3 6 0 0 0]
 [3 7 8 0 0]
 [9 6 1 0 0]
 [6 4 4 2 8]]

H = 15


In [14]:
# 3. 対称行列の場合
Q_sym = Q_upper + Q_upper.T
print(f"Q_sym = \n{Q_sym}\n")

H_sym = int(np.dot(np.dot(x, Q_sym), x))
print(f"H = {H_sym}")

Q_sym = 
[[12  3  3  9  6]
 [ 3 12  7  6  4]
 [ 3  7 16  1  4]
 [ 9  6  1  0  2]
 [ 6  4  4  2 16]]

H = 30


以上より，QUBOモデルは
1. 上三角行列
2. 下三角行列
3. 対称行列

のどれで表現してもよい．  
ただし，対称行列で表現するとハミルトニアンの値は2倍になる．  
2倍になるが，各状態におけるハミルトニアンも2倍になる(要は，解空間全体が2倍にシフトするだけで，基底状態は変わらない)ので問題はない．

## 非対称QUBOモデル
扱う問題によっては，稀にQUBOモデルが非対称行列となることがある．  
例えば，[QAPLIB](https://www.opt.math.tugraz.at/qaplib/inst.html)の[Bur26a](https://www.opt.math.tugraz.at/qaplib/data.d/bur26a.dat)インスタンスは，QUBOモデルに落とし込むと非対称QUBO行列が生成される．  
ここでは，非対称QUBOモデルについて考えてみる．

In [15]:
# ========== 非対称行列の場合 ==========
Q_asym = np.random.randint(coef_min, coef_max+1, size=(num_spin, num_spin))
print(f"Q_asym = \n{Q_asym}\n")

H_asym = int(np.dot(np.dot(x, Q_asym), x))
print(f"H = {H_asym}")

Q_asym = 
[[3 2 0 3 5]
 [6 1 4 3 6]
 [8 4 3 1 3]
 [5 6 9 0 7]
 [4 6 2 8 9]]

H = 11


非対称QUBO行列を$Q_{\mathrm{asym}}$，対称QUBO行列を$Q_{\mathrm{sym}}$とすると，  
$$
Q_{\mathrm{sym}} = \left( Q_{\mathrm{asym}}の上三角行列 \right) + \left( Q_{\mathrm{asym}}の下三角行列 \right)^T - \left( Q_{\mathrm{asym}}の対角成分 \right)
$$
とすることで，$Q_{\mathrm{asym}}$に等価な$Q_{\mathrm{sym}}$を生成することができる．

In [16]:
Q_sym = np.triu(Q_asym) + np.tril(Q_asym).T - np.diag(Q_asym.diagonal())
print(f"Q_sym = \n{Q_sym}\n")

H_sym = int(np.dot(np.dot(x, Q_sym), x))
print(f"H = {H_sym}")

Q_sym = 
[[ 3  8  8  8  9]
 [ 0  1  8  9 12]
 [ 0  0  3 10  5]
 [ 0  0  0  0 15]
 [ 0  0  0  0  9]]

H = 11
